In [1]:
import pandas as pd

In [30]:
github_url='https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [42]:
df = df.head(300)

In [43]:
answer_llm=df.iloc[0].answer_llm

In [44]:
from sentence_transformers import SentenceTransformer
model_name='multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

In [ ]:
v = embedding_model.encode(answer_llm)

Question 1

In [115]:
v[0]

-0.42244658

In [64]:
from tqdm.auto import tqdm
evaluations_llm = []
evaluations_orig = []

for i, doc in tqdm(df.iterrows()):
        answer_llm = doc['answer_llm']
        answer_orig = doc['answer_orig']
        evaluations_llm.append(embedding_model.encode(answer_llm))
        evaluations_orig.append(embedding_model.encode(answer_orig))

0it [00:00, ?it/s]

In [71]:
scores = [np.dot(e1,e2) for e1, e2 in zip(evaluations_llm, evaluations_orig)]

Question 2:

In [72]:
np.percentile(scores, 75)

31.674312114715576

In [74]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [87]:
from tqdm.auto import tqdm
evaluations_llm_norm = []
evaluations_orig_norm = []

for i, doc in tqdm(df.iterrows()):
        answer_llm = doc['answer_llm']
        answer_orig = doc['answer_orig']
        evaluations_llm_norm.append(normalize(embedding_model.encode(answer_llm)))
        evaluations_orig_norm.append(normalize(embedding_model.encode(answer_orig)))

0it [00:00, ?it/s]

In [88]:
cosine_score = [np.dot(e1,e2) for e1, e2 in zip(evaluations_llm_norm, evaluations_orig_norm)]

Question 3:

In [89]:
np.percentile(cosine_score, 75)

0.8362347930669785

In [91]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [96]:
from rouge import Rouge
rouge_scorer = Rouge()

rouge_scores = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])[10]

Question 4:

In [116]:
rouge_scores['rouge-1']['f']

0.45454544954545456

Question 5:

In [110]:
mean_rouge_score = (rouge_scores['rouge-1']['f'] + rouge_scores['rouge-2']['f'] + rouge_scores['rouge-l']['f']) / 3
mean_rouge_score

0.35490034990035496

In [112]:
from tqdm.auto import tqdm
results = []

for i, doc in tqdm(df.iterrows()):
    rouge_score = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])[i]
    results.append({
        'rouge1': rouge_score['rouge-1']['f'],
        'rouge2': rouge_score['rouge-2']['f'],
        'rougel': rouge_score['rouge-l']['f'],
    })

0it [00:00, ?it/s]

Question 6:

In [114]:
r_df = pd.DataFrame(results)
mean_rouge2 = r_df['rouge2'].mean()
mean_rouge2

0.20696501983423318